<a href="https://colab.research.google.com/github/chhammet/ST554_HW4/blob/main/Task3/project1_task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 1: Task 3
Author: Cole Hammett

Collaborators: Bing Chen, Cole Hammett

Course: ST554 Analysis of Big Data

Instructor: Justin Post

Date: 25th of February, 2026

Purpose: Project 1-Task 3

# Introduction

## Task 3 Goal
To evaluate two linear regression models using time-series-aware cross-validation on benzene estimation in an urban pollution monitoring scenario.

## Step 1: Loading and Cleaning the Data

### Importing libraries

Used for data cleaning, wrangling, and EDA

In [110]:
import pandas as pd
import numpy as np

Used for data visualization

In [111]:
import matplotlib.pyplot as plt

Used for modeling

In [112]:
from sklearn import linear_model
import sklearn.metrics as metrics

Installing UCI machine learning repository library to access data

In [113]:
!pip install ucimlrepo

## Load the Data

Loading newly installed library

In [114]:
import ucimlrepo as uci

Fetching data from UCI machine learning repository library

In [115]:
air_quality = uci.fetch_ucirepo(id=360)

## Preliminary Viewing

Done to check the status of the data before cleaning it. Checking for the size and shape of the data. Looking for errors, nulls, or other missing values.

Assigning data to a variable to improve accessibility

In [116]:
aq_df = air_quality.data.features

Checking the data type

In [117]:
print(type(aq_df))

<class 'pandas.core.frame.DataFrame'>


Data type is a pandas DataFrame. This type of object is used to handle rectangular datasets

Viewing the first few rows with `.head()` method, default to first 5

In [118]:
aq_df.head()

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,3/10/2004,18:00:00,2.6,1360,150,11.9,1046,166,1056,113,1692,1268,13.6,48.9,0.7578
1,3/10/2004,19:00:00,2.0,1292,112,9.4,955,103,1174,92,1559,972,13.3,47.7,0.7255
2,3/10/2004,20:00:00,2.2,1402,88,9.0,939,131,1140,114,1555,1074,11.9,54.0,0.7502
3,3/10/2004,21:00:00,2.2,1376,80,9.2,948,172,1092,122,1584,1203,11.0,60.0,0.7867
4,3/10/2004,22:00:00,1.6,1272,51,6.5,836,131,1205,116,1490,1110,11.2,59.6,0.7888


Using `.shape` attribute to view the dimensions of the DataFrame

In [119]:

aq_df.shape

(9357, 15)

Using `.info()` method to view information about the data frame

In [120]:
aq_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9357 entries, 0 to 9356
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           9357 non-null   object 
 1   Time           9357 non-null   object 
 2   CO(GT)         9357 non-null   float64
 3   PT08.S1(CO)    9357 non-null   int64  
 4   NMHC(GT)       9357 non-null   int64  
 5   C6H6(GT)       9357 non-null   float64
 6   PT08.S2(NMHC)  9357 non-null   int64  
 7   NOx(GT)        9357 non-null   int64  
 8   PT08.S3(NOx)   9357 non-null   int64  
 9   NO2(GT)        9357 non-null   int64  
 10  PT08.S4(NO2)   9357 non-null   int64  
 11  PT08.S5(O3)    9357 non-null   int64  
 12  T              9357 non-null   float64
 13  RH             9357 non-null   float64
 14  AH             9357 non-null   float64
dtypes: float64(5), int64(8), object(2)
memory usage: 1.1+ MB


## Clean the data

Counting the missing values to consider the proportion of data to be dropped

In [121]:
obs_var = ['C6H6(GT)', 'CO(GT)', 'T', 'RH', 'AH']

In [122]:
for col in obs_var:
    count = (aq_df[col] == -200).sum()
    print(f"Column '{col}': {count} rows contain -200")

Column 'C6H6(GT)': 366 rows contain -200
Column 'CO(GT)': 1683 rows contain -200
Column 'T': 366 rows contain -200
Column 'RH': 366 rows contain -200
Column 'AH': 366 rows contain -200


Removing observations where any of the five variables are -200

In [123]:
for col in obs_var:
    aq_df = aq_df[aq_df[col] != -200]

Checking that it worked

In [124]:
for col in obs_var:
    count = (aq_df[col] == -200).sum()
    print(f"Column '{col}': {count} rows contain -200")

Column 'C6H6(GT)': 0 rows contain -200
Column 'CO(GT)': 0 rows contain -200
Column 'T': 0 rows contain -200
Column 'RH': 0 rows contain -200
Column 'AH': 0 rows contain -200


Subsetting down to only necessary variables

In [125]:
aq_df[['Date', 'C6H6(GT)', 'CO(GT)', 'T', 'RH', 'AH']]


,Date,C6H6(GT),CO(GT),T,RH,AH
0,3/10/2004,11.9,2.6,13.6,48.9,0.7578
1,3/10/2004,9.4,2.0,13.3,47.7,0.7255
2,3/10/2004,9.0,2.2,11.9,54.0,0.7502
3,3/10/2004,9.2,2.2,11.0,60.0,0.7867
4,3/10/2004,6.5,1.6,11.2,59.6,0.7888
...,...,...,...,...,...,...
9352,4/4/2005,13.5,3.1,21.9,29.3,0.7568
9353,4/4/2005,11.4,2.4,24.3,23.7,0.7119
9354,4/4/2005,12.4,2.4,26.9,18.3,0.6406
9355,4/4/2005,9.5,2.1,28.3,13.5,0.5139


Finding the averages for each variable for all time points within a day

In [126]:
aq_df.groupby("Date")[obs_var].mean().sort_values(by='Date', ascending = False)

,C6H6(GT),CO(GT),T,RH,AH
Date,,,,,
9/9/2004,13.541667,2.237500,28.525000,26.550000,0.964671
9/8/2004,17.016667,3.316667,27.966667,26.433333,0.965850
9/7/2004,9.700000,1.890909,27.618182,33.318182,1.154682
9/6/2004,6.278261,1.421739,26.752174,35.934783,1.219478
9/5/2004,5.712500,1.304167,29.204167,27.237500,1.057429
...,...,...,...,...,...
1/13/2005,12.495833,2.679167,9.991667,69.566667,0.847225
1/12/2005,15.817391,3.273913,12.021739,65.443478,0.904865
1/11/2005,13.779167,2.812500,12.779167,64.104167,0.941413


# Step 2: Model Definitions and Helper Functions

## Create data to reference for time-series cross-validation

Sorting dataframe by date

In [127]:
aq_df = aq_df.sort_values('Date').reset_index(drop=True)

Assigning an integer value to each position

In [128]:
day_arr = pd.factorize(aq_df['Date'])[0]

## SLR model using `CO(GT)` to predict `C6H6(GT)`

SLR = Simple Linear Regression

*   Input: 1D response $y$ and a single predictor, both numeric.
*   Output: Estimates of slope and intercept of a line that can be used to predict a response value from a given predictor value.

$$Y_i = b_0 + b_1x_i + E_i$$

Response (C6H6)  = intercept + slope * predictor (CO) + Error



Setting up predictor variable

In [129]:
slr_x = aq_df['CO(GT)'].values.reshape(-1,1)

In [130]:
slr_x.shape

(7344, 1)

Setting up response variable

In [131]:
slr_y = aq_df['C6H6(GT)'].values

In [132]:
slr_y.shape

(7344,)

## MLR model using `CO(GT)`, `T`, `RH`, and `AH` to predict `C6H6(GT)`

MLR = Multiple Linear Regression Model

*   Input: 1D response $y$ and more than two predictor, all numeric in this case.
*   Output: Estimates of **slopes** and intercept of a line that can be used to predict a response value from multiple predictor values.

For this project, the model will be:

$$E(Y|x) = b_0 + b_1x_1 + b_2x_2 + b_3x_3 + b_4x_4$$

Where

$Y$ is the value of the response variable (C6H6 concentration) at some observation $x$

$x_1$ is the concentration of CO

$x_2$ is the temperature

$x_3$ is the relative humidity

$x_4$ is the absolute humidity

This model is appropriate within the scope of this project as the aim is to design and implement a cross-validation scheme for time-series data. This is an easy to use model as it does not include any interaction effects or polynomials.

Setting up predictor variable

In [133]:
mlr_x = aq_df[['CO(GT)', 'T', 'RH', 'AH']]

Setting up response variable

In [134]:
mlr_y = aq_df['C6H6(GT)']


## Training and Evaluation Method:

**Ordinary least squares regression**

* We will fit a linear model with intercept $b_0$ and coefficient(s) $b_1, ... , b_n$ that minimizes the mean of squared error between the observed targets in the dataset (the training data), and the targets predicted by the linear approximation (the test data).
* The loss function used for model fitting and evaluation is mean of squared error.

$$MSE = \frac{1}{n}\sum_{i=1}^{n}(y_i - \hat{y}_i)^2$$

* This value is calculated by finding the difference between the actual value ($y_i$) and the predicted value ($\hat{y}_i$) then squaring that difference. These are summed then divided by the total number of observations to find the mean.

* The values of the slope can be calculated with:

$$b_1 = \frac{\sum_{i=1}^{n} (x_i - \bar{x})(y_i - \bar{y})}{\sum_{i=1}^{n} (x_i - \bar{x})^2}$$


* The values of the intercept can be calculated with:

$$b_0 = \bar{y} - b_1\bar{x}$$




## A helper function for calculating MSE

In [135]:
def get_one_step_mse(x, y, day, day_arr):

  """
  Calculate the mean squared error for a cross-validation of
  training on all variable data leading up a date then testing on the
  day immediately after that.

  Parameters:
  x (numpy.ndarray): An array of predictor variables.
  y (numpy.ndarray): An array of response variables.
  day (int): The day in the time series to use for testing.
  day_arr (numpy.ndarray): An array of time positions in the time series.

  Returns:
  float: The mean squared error for the given day.

  """

  # train through day i
  train_mask = day_arr <= day

  # test on day i+1
  test_mask  = day_arr == (day + 1)

  # create cv splits using masks
  x_train, y_train = x[train_mask], y[train_mask]
  x_test,  y_test  = x[test_mask],  y[test_mask]

  # fit model on training set
  reg = linear_model.LinearRegression()
  reg.fit(x_train, y_train)

  # evaluate model on test set
  pred = reg.predict(x_test)

  # calculate evaluation metric
  return metrics.mean_squared_error(y_test, pred)

## A helper function for cross-validation

In [136]:
def get_cv_mse(x, y, day, day_arr):

  """
  Calculate the mean squared error for a cross-validation of
  training on all variable data leading up a date

  Parameters:
  x (numpy.ndarray): An array of predictor variables.
  y (numpy.ndarray): An array of response variables.
  day (int): The day in the time series to use for splitting.
  day_arr (numpy.ndarray): An array of time positions in the time series.

  Returns:
  float: The mean squared error for all folds

  """
  # initialize MSE at 0
  mse = 0

  # last available day index
  max_day = day_arr.max()

  # loop from day to max(Day) - 1
  for i in range(day, max_day):

  # call the helper function, sum with augmented assignment of result
    mse += get_one_step_mse(x, y, i, day_arr)

  # calculate average MSE across all folds
  return mse / (max_day - day)


# Step 3: Model Comparison and Results

## Calculate MSEs during cross-validation for SLR and MLR

* During each fold, the model is predicting a single value for the unseen timepoint (the testing set) which immediately sucedes the last timepoint used to fit the model (the training set).
* Because this is supervised learning, the ACTUAL value of the timepoint being predicted on is known. Thus, we can calculate the error (difference between the actual and the predicted values).
* To evaluate model performance, we calculate sqaured error for each fold then find the mean accross all folds.

In [137]:
slr_mse = get_cv_mse(slr_x, slr_y, 250, day_arr)
print(slr_mse)

6.271434434680725


In [138]:
mlr_mse = get_cv_mse(mlr_x, mlr_y, 250, day_arr)
print(mlr_mse)

4.080432244571008


The fitted MLR model has a lower MSE than the fitted SLR model when evaluating on a test set of unseen data at a future timepoint. This result is consistent with the findings of the project that the data was sourced from. Including meteorlogical data like temperature, relative humidity, and absolute humidity would seem to improve model performance as "Results are affected by what
we expect to be seasonal meteorological effects..." (De vito et al., 2008).

## Fit the best model

Creating a class object `reg` to create instances of LinearRegression, a class from sklearn/linear_model.

In [139]:
reg = linear_model.LinearRegression()

Using the `.fit()` method to fit the model, then view the intercept and coefficients

In [144]:
reg.fit(mlr_x, mlr_y)

print(reg.intercept_, reg.coef_)

-2.3273284432631502 [ 4.80090314  0.1030811  -0.00847744  0.97289547]
